In [1]:
ip = 'products.igs-ip.net'
port = 2101
mountpoint = 'BCEP00BKG0'
user = 'Hcheong'
password = 'Gnss12345'

# connect to this stream

import socket
import sys
import time
import datetime
import base64
from pathlib import Path

def get_data(ip, port, mountpoint, user, password):
    # Encode credentials in base64 for HTTP Basic Auth
    credentials = base64.b64encode(f"{user}:{password}".encode()).decode()

    # Create a socket object
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((ip, port))
        # Send HTTP GET request with Authorization header
        request = f"GET /{mountpoint} HTTP/1.0\r\nHost: {ip}\r\nAuthorization: Basic {credentials}\r\n\r\n"
        s.sendall(request.encode())
        
        # Receive the data
        data = b''
        try:
            while True:
                chunk = s.recv(1024)
                if not chunk:
                    break
                data += chunk
        except Exception as e:
            print(f"Error receiving data: {e}")
        
    return data

data = get_data(ip, port, mountpoint, user, password)
print(data)

b'HTTP/1.1 403 Forbidden\r\nServer: NTRIP BKG Caster 2.0.45/2.0\r\nDate: Fri, 19 Apr 2024 12:09:51 GMT\r\n\r\n'


In [2]:
eph_columns = [
     'af0',
 'af1',
 'af2',
 'cic',
 'cis',
 'code_L2',
 'crc',
 'crs',
 'cuc',
 'cus',
 'dn',
 'ecc',
 'fullweek',
 'gnss',
 'gnss_short',
 'gpstoe',
 'gpsweek',
 'health',
 'i0',
 'idot',
 'interval',
 'iodc',
 'iode',
 'm0',
 'omega',
 'omega_0',
 'omega_dot',
 'root_a',
 'sat_id',
 'tgd',
 'toc',
 'toe',
 'ura',
 'week'
]

In [3]:
# download eph from chc stream
from RTCM3_decode.rtcm3_reader import decoderaw, rtcm3
from RTCM3_decode.EPH import ephData
from datetime import datetime

fixed_data_len = 4096 # downloading msg lenth every time
EPHIP = '18.170.8.65'
EPHPORT = 10011

class tcprecv_data:
    def __init__(self,IP,PORT):
        self.IP = IP
        self.PORT = PORT
        self.tcp = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        
    def connect_tcp(self):
        try:
            self.tcp.connect((self.IP, self.PORT))
        except:
            print ('Warning tcp is down from', self.IP,self.PORT)
        return
    
    def recv(self,sec):
        msg = b'' 
        tstart = time.perf_counter()
        try:
            while True:
                #self.tcp.settimeout(1.0)
                data = self.tcp.recv(fixed_data_len)
                msg += data
                tend = time.perf_counter()
                tdiff = tend - tstart
                # sec =1 for corrections
                # sec = 4 for EPH
                if(tdiff > sec): 
                    #tstart = time.perf_counter()
                    break

        except KeyboardInterrupt:
            sys.exit()
        return msg

    def run(self,sec):
    # input: tcp, connected
    # Output: msg from data_downloading in 1 sec     
        msg = b'' 
        try:
            msg = self.recv(sec)
        except KeyboardInterrupt:
            sys.exit()

        return msg
    
    def reconnect(self,sec):
        msg = b'' 
        try:
            self.tcp.close()
            self.connect_tcp()
            msg = self.recv(sec)

        except KeyboardInterrupt:
            sys.exit()
        return msg

class EPH_process:
    def __init__(self,EPHIP,EPHPORT):
        self.EPHIP = EPHIP
        self.EPHPORT = EPHPORT
        self.system ='G'
        self.rtcmeph = rtcm3()
        self.ephUser = ephData.ephUser() # Multiple IODE for each sat
        self.tcp     = tcprecv_data(self.EPHIP,self.EPHPORT)
        self.tcp.connect_tcp()

    def run(self):
        msgEPH = self.tcp.run(sec=5)#
        decoded_data =  decoderaw(self.rtcmeph,msgEPH) #ephdata in ret[1]
        return decoded_data
    
downloader = EPH_process(EPHIP, EPHPORT)
decoded_data = downloader.run()

Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw
Warning in decoding_type1042
Warning in decoderaw


In [11]:
import numpy as np

gps_eph_data = decoded_data[1].ephgps

def extract_attributes(data):
    # Initialize an empty dictionary to store attribute values
    attributes = {}
    
    # Loop through each attribute in the dir() of the data object
    for attr in dir(data):
        # Ignore private and built-in attributes/methods
        if not attr.startswith('__'):
            # Try to get the value of the attribute and add it to the dictionary
            try:
                value = getattr(data, attr)
                attributes[attr] = value
            except AttributeError:
                print(f"Attribute {attr} could not be accessed")
    
    return attributes

data_list = []
for i in range(32):

    try:
        gps_prn_data = gps_eph_data[i]
        gps_prn_data = extract_attributes(gps_prn_data)
        
        eph_parameters = {
        'obs_unix_timestamp':datetime.now().timestamp(), 
        'system':1,
        # 'prn': int(gps_prn_data.sat_id[1:]),
        'bias': np.nan,
        'drift': np.nan,
        'driftRate': np.nan ,
        'iode': gps_prn_data.iode,
        'crs': gps_prn_data.crs,
        'delta_n':gps_prn_data.dn ,
        'mean_anomaly_0': gps_prn_data.m0, 
        'cuc': gps_prn_data.cuc, 
        'e': gps_prn_data.ecc, 
        'cus': gps_prn_data.cus, 
        'sqrt_a': gps_prn_data.root_a ,
        'toe_sow': gps_prn_data.toe ,
        'cic': gps_prn_data.cic ,
        'omega_0': gps_prn_data.omega_0 ,
        'cis': gps_prn_data.cis ,
        'i_0': gps_prn_data.i0 ,
        'crc': gps_prn_data.crc ,
        'omega': gps_prn_data.omega ,
        'omega_dot': gps_prn_data.omega_dot,
        'i_dot' : gps_prn_data.idot
        }

        data_list.append(eph_parameters)
    except:
        data_list.append({})

gps_eph_arr = np.vstack([list(i.values()) for i in data_list])

{}
{'L2P': 0, 'af0': -0.00045075174421072006, 'af1': 6.480149750132114e-12, 'af2': 0.0, 'cic': -5.289912223815918e-07, 'cis': -2.4400651454925537e-07, 'code_L2': 0, 'crc': 189.8125, 'crs': 7.34375, 'cuc': 3.501772880554199e-07, 'cus': 1.0423362255096436e-05, 'dn': 4.071598169835356e-09, 'ecc': 0.01601860870141536, 'fullweek': None, 'gnss': 'GPS', 'gnss_short': 'G', 'gpstoe': 471600, 'gpsweek': 262, 'health': 0, 'i0': 0.9676251773354494, 'idot': -3.8215877558980976e-11, 'interval': -1, 'iodc': 104, 'iode': 104, 'm0': -0.22038820780204388, 'omega': -1.2013416777315211, 'omega_0': 2.6875980577725858, 'omega_dot': -7.988547040763809e-09, 'root_a': 5153.705785751343, 'sat_id': 'G02', 'tgd': -1.7695128917694092e-08, 'toc': 471600, 'toe': 471600, 'ura': 2, 'week': 262}
{'L2P': 0, 'af0': 0.0003190981224179268, 'af1': 2.0691004465334117e-11, 'af2': 0.0, 'cic': 4.6566128730773926e-08, 'cis': -5.587935447692871e-09, 'code_L2': 1, 'crc': 294.8125, 'crs': 31.46875, 'cuc': 1.6540288925170898e-06, 'c

In [ ]:
from GNSS import GPSKepler

gps_kepler = GPSKepler(gps_eph_arr)
sv_pos_xyz = gps_kepler.run()

print(sv_pos_xyz)

[[ 1.71345142e+09  1.00000000e+00  1.00000000e+00  1.05930786e+07
  -2.35360330e+07  5.12484790e+06]
 [ 1.71345142e+09  1.00000000e+00  2.00000000e+00  1.39942849e+07
  -1.25104588e+07  1.91624929e+07]
 [ 1.71345142e+09  1.00000000e+00  3.00000000e+00  2.27073347e+07
  -1.26810218e+07 -5.20177361e+06]
 [ 1.71345142e+09  1.00000000e+00  4.00000000e+00  1.50973085e+07
  -6.25062443e+06 -2.09480991e+07]
 [ 1.71345142e+09  1.00000000e+00  5.00000000e+00 -2.46269200e+07
   4.18417858e+06 -9.33261270e+06]
 [ 1.71345142e+09  1.00000000e+00  6.00000000e+00 -6.28611478e+06
  -2.11987757e+07 -1.45665268e+07]
 [ 1.71345142e+09  1.00000000e+00  7.00000000e+00  5.24943052e+06
  -2.56436805e+07 -2.06015014e+06]
 [ 1.71345142e+09  1.00000000e+00  8.00000000e+00  2.14148229e+07
   1.88858065e+06  1.57609263e+07]
 [ 1.71345142e+09  1.00000000e+00  9.00000000e+00  2.55304379e+06
  -1.63876280e+07 -2.08132512e+07]
 [ 1.71345142e+09  1.00000000e+00  1.00000000e+01  5.55989766e+06
   1.37287434e+07  2.2253

In [ ]:
from Coord import ECEF


sat_pos_ecef = ECEF(sv_pos_xyz[:,3:])
sat_pos_lla = sat_pos_ecef.ecef2lla()
a = sat_pos_lla.position